In [ ]:
import json
import numpy as np
import pandas
import string
from pprint import pprint
from datetime import datetime, timedelta
import time
import os
pandas.options.display.max_colwidth = -1
data_dir = '/home/idies/workspace/Temporary/raddick/baseball_scratch/'
print('ok')

In [ ]:
allgames_df = pandas.DataFrame()
for i in range(2010,2018):
    filename = '/home/idies/workspace/Temporary/raddick/baseball_scratch/events_by_year/events_{0:.0f}.json'.format(i)
    s = time.time()
    with open(filename,'r') as f:
        games = json.load(f)
    e = time.time()
    print('{0:.0f}: Loaded JSON file with events data: {1:,.0f} games found in {2:,.0f} seconds...'.format(i, len(games), e-s))
    print('Changing data types and concatenating...')
    for j in range(0,676):#,len(games)):
        #if ((np.mod(j,500) == 0) | (j >= len(games)-1)):
        #print('Game {0:,.0f} of {1:,.0f}...'.format(j, len(games)-1))
        thisgame_df = pandas.DataFrame.from_records(games[j]['info'], index=[games[j]['id']])
        thisgame_df.index.name = 'gameID'
        #pprint(thisgame_df)
        
        # Parse datetime variables
        thisgame_df['date'] = pandas.to_datetime(thisgame_df['date'], format='%Y/%m/%d', errors='coerce')
        thisgame_df['timeofgame'] = pandas.to_numeric(thisgame_df['timeofgame'], errors='coerce').dropna().apply(lambda x: timedelta(minutes=x))
        try:
            thisgame_df['starttime'] = pandas.to_datetime(thisgame_df['starttime'], format='%I:%M%p', errors='coerce')
        except:
            thisgame_df['starttime'] = pandas.to_datetime(pandas.NaT)        
        
        # Parse numerics
        try:
            thisgame_df['attendance'] = pandas.to_numeric(thisgame_df['attendance'], errors='coerce')
        except KeyError:
            print('{0:}: {1:} not found'.format(thisgame_df.index.get_values()[0],'attendance'))
        try:
            thisgame_df['windspeed'] = pandas.to_numeric(thisgame_df['windspeed'], errors='coerce')
        except KeyError:
            print('{0:}: {1:} not found'.format(thisgame_df.index.get_values()[0],'windspeed'))
            #thisgame_df['windspeed'] = -1
        try:
            thisgame_df['number'] = pandas.to_numeric(thisgame_df['number'], errors='coerce')
        except KeyError:
            #thisgame_df['number'] = -1
            print('{0:}: {1:} not found'.format(thisgame_df.index.get_values()[0],'number'))
        try:
            thisgame_df['temp'] = pandas.to_numeric(thisgame_df['number'], errors='coerce')
        except KeyError:
            #thisgame_df['temp'] = -1
            print('{0:}: {1:} not found'.format(thisgame_df.index.get_values()[0],'temp'))

        # remove quotes
        for thiscol in ['inputprogvers', 'inputter', 'scorer', 'translator']:
            try:
                thisgame_df[thiscol] = thisgame_df[thiscol].apply(lambda x: x[x.find('"')+1:x.find('""')])
            except KeyError:
                thisgame_df[thiscol] = 'n/a'
        #        print('{0:}: {1:} not found'.format(thisgame_df.index.get_values()[0],thiscol))
        allgames_df = pandas.concat((allgames_df, thisgame_df), sort=False)
    
#for onegame in games:
#    print(onegame['id'])
#    pprint(onegame['info'])

print('Parsed {0:,.0f} games, writing output as csv...'.format(len(allgames_df)))
allgames_df.to_csv('/home/idies/workspace/Temporary/raddick/baseball_scratch/gameinfo_from_events_2010s.csv')
print('Output written!')

print('Done!')


In [ ]:
columns_to_add

In [ ]:
s = time.time()
allgames_df = pandas.DataFrame()

#for i in range(0, len(games)):
for i in range(100000, 120000):
    if ((np.mod(i, 5000) == 0) | (i==len(games)-1)):
        print('Reading game number {0:,.0f}...'.format(i))
    game_df = pandas.DataFrame.from_records(games[i]['info'], index=[games[i]['id']])
    allgames_df = pandas.concat((allgames_df, game_df), sort=False)
e = time.time()

print('Loaded {0:,.0f} games in {1:,.0f} seconds.'.format(len(allgames_df), e-s))



#allgames_df

## Prior code that had accounted for derived games

In [ ]:
allgames = []
yeararray = [1921, 1922, 1925, 1927, 1928]
for i in np.linspace(1930,2016,87):
    yeararray.append(i)
#yeararray = [2016]
    
years_with_derived = []
for i in np.linspace(1942,1959,18):
    years_with_derived.append(i)
for i in np.linspace(1962,1968,7):
    years_with_derived.append(i)
for i in np.linspace(1971,1973,3):
    years_with_derived.append(i)
print('years identified')
    
for thisyear in yeararray:
#    print('Starting to parse the year {0:.0f}'.format(thisyear))
    filename = 'raw/events_raw_{0:.0f}.json'.format(thisyear)
    print(filename)
    with open(filename,'r') as f:
        games = json.load(f)
    if (thisyear in years_with_derived):
        derived_filename = 'with_derived/events_with_derived_{0:.0f}.json'.format(thisyear)
        print(derived_filename)
        with open(derived_filename,'r') as f:
            games_derived = json.load(f)
        games = games + games_derived
    allgames = allgames + games

#    games_df = pandas.DataFrame(games)
#    games_df = games_df.set_index('id')
    print('{0:,.0f} games found'.format(len(allgames)))

    allgames_df = pandas.DataFrame()
    for i in range(0, len(allgames)):
        if ((np.mod(i, 100) == 0) | (i==len(allgames)-1)):
            print('Reading game number {0:,.0f}'.format(i))
        game_df = pandas.DataFrame.from_records(allgames[i]['info'], index=[allgames[i]['id']])
        allgames_df = pandas.concat((allgames_df, game_df))
    allgames_df.index.name = 'gameID'

    allgames_df['attendance'] = pandas.to_numeric(allgames_df['attendance'], errors='coerce')
    allgames_df['date'] = pandas.to_datetime(allgames_df['date'], format='%Y/%m/%d', errors='coerce')
    allgames_df['number'] = pandas.to_numeric(allgames_df['number'], errors='coerce')
    allgames_df['starttime'] = pandas.to_datetime(allgames_df['starttime'], format='%I:%M%p', errors='coerce')
    allgames_df['temp'] = pandas.to_numeric(allgames_df['temp'], errors='coerce')
    allgames_df['timeofgame'] = pandas.to_numeric(allgames_df['timeofgame'], errors='coerce').dropna().apply(lambda x: timedelta(minutes=x))
    allgames_df['windspeed'] = pandas.to_numeric(allgames_df['windspeed'], errors='coerce')
    print('finished reading data')
    #allgames_df.tail(3).transpose()
    allgames_df.to_csv('games/gameinfo_from_events_{0:.0f}.csv'.format(thisyear))
    print('output written')

In [ ]:
# runs in about 5 minutes
with open('events_with_derived.json','r') as f:
    allgames = json.load(f)

allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')

print('done')

In [ ]:
info_df = pandas.DataFrame.from_records(allgames_df['info'], index=allgames_df.index)

column_list = ['visteam', 'hometeam', 'date', 'number', 'starttime', 'daynight']
column_list += ['site', 'timeofgame', 'attendance']
column_list += ['temp', 'fieldcond', 'precip', 'sky', 'winddir', 'windspeed']
column_list += ['umphome', 'ump1b', 'umb2b', 'umb3b', 'umplf', 'umprf']
column_list += ['wp', 'lp', 'save']
column_list += ['how_scored', 'oscorer', 'pitches', 'usedh']

#info_df.columns = column_list

#number -> gameno
#timeofgame -> length

info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
info_df['number'] = pandas.to_numeric(info_df['number'], errors='coerce')
info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
info_df['timeofgame'] = pandas.to_timedelta(info_df['timeofgame'].astype(str) + ' minutes', unit='m', errors='coerce')
info_df = info_df.fillna('')

info_df.to_csv('gameinfo_from_events.csv')
print('Done')